In [ ]:
import pandas as pd

data = pd.read_csv("data/Hawthorne/Site_3006_cleanded.csv", index_col=0)

data.head()


## Calculating K
- Convert Temp to K
- Calculate K

In [ ]:
# Coversion
data['Temp[K]'] = (data['Temp'] - 32) * 5/9 + 273.15

# K calc
A = 3.0e-12
EoR = 1500
data['K'] = A * np.exp(-EoR / (data['Temp[K]'])) #This unit seems to be cm**3 Molecules**-1 s**-1

## Calculating J
- I am slightly skeptical about this one because the instrament we got the SR from this time seems to be different 

In [ ]:
Data['SR W m-1'] = Data['SR'] * 697.3
C=7.784e-5
Data['J'] = c * Data['SR W m-1'] 
